In [1]:
# import sys
# sys.path.append('home/Documents/code/github/motile_tracker/') #maybe needed for debugging??


import numpy as np
import pandas as pd
import dask.array as da
import matplotlib.pyplot as plt
import napari
import shutil

from pathlib import Path
# from typing import Dict, List, Tuple, Optional, Iterable,TypeAlias, Mapping, Sequence
# from pyinstrument import Profiler

from motile_tracker.data_model.actions import AddEdges, DeleteNodes, DeleteEdges, AddNodes

from functions.DatabaseArray import DatabaseArray
# from functions.get_value_by_field import get_value_by_field
from functions.TrackEdit_functions import *
from functions.database_handler import *
from functions.motile_overwrites import *

Make segments+graph

In [2]:
#**********INPUTS*********
working_directory = Path('/home/teun.huijben/Documents/data/Akila/20241003/neuromast4_t851/adjusted/')
db_filename_old = 'data.db'
time_window = (0,100)
data_shape_full = [851,73,1024,1024]      #T,(Z),Y,X       (851,73,1024,1024)
scale = (4,1,1)
layer_name = 'ultrack'
allow_overwrite = True      #overwrite existing database/changelog
#*************************

#handle filenames of db/changelog

DB_handler = database_handler(
                db_filename_old= db_filename_old,
                working_directory= working_directory,
                data_shape_full= data_shape_full,
                z_scale = scale[0],
                allow_overwrite= allow_overwrite)

#db > tracks > nxgraph
nxgraph = DB_handler.nxgraph
#db > segments
segments = DB_handler.segments

print('\nsegments',segments.shape)
# print('type(segments)',type(segments))

shapes full/chunk/length [851, 73, 1024, 1024] [100, 73, 1024, 1024] 100 8
Database copied to: /home/teun.huijben/Documents/data/Akila/20241003/neuromast4_t851/adjusted/data_v1.db
old database: data.db
new database: data_v1.db
new logfile: data_v1_changelog.txt

segments [100, 73, 1024, 1024]


Overwrite motile functions

In [3]:
DeleteNodes._apply = create_db_delete_nodes(DB_handler)
DeleteEdges._apply = create_db_delete_edges(DB_handler)
AddEdges._apply = create_db_add_edges(DB_handler)
AddNodes._apply = create_db_add_nodes(DB_handler)
TracksViewer._refresh = create_tracks_viewer_and_segments_refresh(layer_name=layer_name)

TrackEdit

In [4]:
viewer = napari.Viewer()

add_motile_widgets(viewer)

add_tracks(viewer, 
           nxgraph = nxgraph, 
           segmentation=segments, 
           name = layer_name)

viewer.dims.ndisplay = 3    #3D view
viewer.layers.selection.active = viewer.layers['ultrack_seg']   #select segmentation layer

Making new tracking view controller


change time chunk

In [7]:

time_window = (0,10)

data_shape[0] = time_window[1]-time_window[0]    #T,(Z),Y,X     

# DB.handler ...

add_tracks(viewer, 
           nxgraph = DB_handler.nxgraph, 
           segmentation=DB_handler.segments, 
           name = layer_name)






In [9]:
tv = TracksViewer.get_instance(viewer)
tv.tracks_list
print(tv.tracks_list)

In [ ]:
# CHECK THE NUMBER OF TRACKS IN TRACKS LIST 

# Playing

Just DatabaseArray in napari

In [ ]:
# viewer = napari.Viewer()
# viewer.add_labels(segments,scale=scale)

Todo:
- add display showing the time
- implement the actual rerendering of a new block
- disable buttons when first/last block is shown 
- ~~put buttons next to each other~~